## 데이터 불러오기

In [264]:
import pandas as pd

data = pd.read_csv('data_dev.txt', sep='~', engine='python', index_col=False)
company_map = pd.read_csv('company_map.txt')

## 데이터 전처리

In [239]:
import re
p = re.compile(r"[^a-zA-Z0-9!&'()*/ .-]")
def remain_vaild_char(string):
    tmp_string = re.sub('\n', ' ', string)
    tmp_string = re.sub(p, '', tmp_string)
    return tmp_string

data['refine_headline'] = data['Headline'].apply(remain_vaild_char)
data['refine_body'] = data['Body'].apply(remain_vaild_char)

In [240]:
# "Press Release" 지우기
data['refine_headline'] = [i.replace("Press Release", "") for i in data.refine_headline]

In [241]:
company_map['split_officialname'] = company_map['OfficialName'].apply(lambda x : x.split())

In [242]:
data['split_body'] = data['refine_body'].apply(lambda x : x.split())
data['split_head'] = data['refine_headline'].apply(lambda x : x.split())

## 빈도수 많은 단어 제거

In [243]:
from itertools import chain

name_partial_list = list(chain.from_iterable(company_map['split_officialname'].values))
body_partial_list = list(chain.from_iterable(data['split_body'].values))
head_partial_list = list(chain.from_iterable(data['split_head'].values))

In [244]:
text_list = body_partial_list+head_partial_list

In [245]:
from collections import Counter
counter1 = Counter(name_partial_list) # 기사명에서 빈도수
counter2 = Counter(text_list) # 기사 제목,본문에서 빈도수

In [246]:
company_high_frequency_word = [pair[0] for pair in counter1.items() if pair[1] >= 10]
text_high_frequency_word = [pair[0] for pair in counter2.items() if pair[1] >= 10]

In [247]:
company_map['refine_officialname'] = company_map['split_officialname'].apply(lambda x : [word for word in x if word not in company_high_frequency_word])
company_map['refine_officialname'] = company_map['refine_officialname'].apply(lambda x : [word for word in x if word not in text_high_frequency_word])

In [248]:
# n-gram
company_map['refine_officialname_add1'] = company_map['split_officialname'].apply(lambda x : ' '.join(x[:2]))
company_map['refine_officialname_add2'] = company_map['split_officialname'].apply(lambda x : ' '.join(x[:3]))
company_map['refine_officialname_add3'] = company_map['split_officialname'].apply(lambda x : ' '.join(x[:4]))
company_map['refine_officialname_add4'] = company_map['split_officialname'].apply(lambda x : ' '.join(x))

In [249]:
#word_dict = {}
#empty_list = []
#for _, row in company_map.iterrows():
#    for word in row['refine_officialname']:
#        tmp = word_dict.get(word, [])
#        tmp.append(row['CompanyID'])
#        word_dict[word] = tmp

## word_dict 만들기

In [255]:
word_dict.get(company_map['refine_officialname_add1'][0],[])

[2094, 2094, 2094, 2094]

In [259]:
word_dict={}
for _, row in company_map.iterrows():
    
    tmp_add1 = word_dict.get(row['refine_officialname_add1'],[])
    tmp_add1.append(row['CompanyID'])
    word_dict[row['refine_officialname_add1']] = tmp_add1
    tmp_add2 = word_dict.get(row['refine_officialname_add2'],[])
    tmp_add2.append(row['CompanyID'])
    word_dict[row['refine_officialname_add2']] = tmp_add2

In [260]:
word_dict

{'INTERSIL CORP-A': [2094, 2094],
 'ARMADA HOFFLER': [3399],
 'ARMADA HOFFLER PROPERTIES': [3399],
 'REPLIGEN CORP': [866, 866],
 'REGIS CORP': [867, 867],
 'ROYAL GOLD': [868],
 'ROYAL GOLD INC': [868],
 'DIGITALGLOBE INC': [1678, 1678],
 'OXFORD SQUARE': [2297],
 'OXFORD SQUARE CAPITAL': [2297],
 'KRAFT HEINZ': [3410],
 'KRAFT HEINZ CO/THE': [3410],
 'KITE PHARMA': [3181],
 'KITE PHARMA INC': [3181],
 'TEXAS ROADHOUSE': [2354],
 'TEXAS ROADHOUSE INC': [2354],
 'VITAL THERAPIES': [2840],
 'VITAL THERAPIES INC': [2840],
 'EXAMWORKS GROUP': [2914],
 'EXAMWORKS GROUP INC': [2914],
 'MANULIFE FINANCIAL': [1978],
 'MANULIFE FINANCIAL CORP': [1978],
 'CORPORATE OFFICE': [869],
 'CORPORATE OFFICE PROPERTIES': [869],
 'GOLDMAN SACHS': [3372, 1830],
 'GOLDMAN SACHS BDC': [3372],
 'CORE MOLDING': [870],
 'CORE MOLDING TECHNOLOGIES': [870],
 'VALERO ENERGY': [3470, 1673],
 'VALERO ENERGY PARTNERS': [3470],
 'STEMLINE THERAPEUTICS': [2902],
 'STEMLINE THERAPEUTICS INC': [2902],
 'BLUE NILE': [201

In [261]:
word_dict = {}
empty_list = []
for _, row in company_map.iterrows():
    tmp_add1 = word_dict.get(row['refine_officialname_add1'],[])
    tmp_add1.append(row['CompanyID'])
    word_dict[row['refine_officialname_add1']] = tmp_add1
    
    tmp_add2 = word_dict.get(row['refine_officialname_add2'],[])
    tmp_add2.append(row['CompanyID'])
    word_dict[row['refine_officialname_add2']] = tmp_add2
    
    tmp_add3 = word_dict.get(row['Ticker'],[])
    tmp_add3.append(row['CompanyID'])
    word_dict[row['Ticker']] = tmp_add3

    tmp_add4 = word_dict.get(row['refine_officialname_add3'],[])
    tmp_add4.append(row['CompanyID'])
    word_dict[row['refine_officialname_add3']] = tmp_add4
    
    tmp_add5 = word_dict.get(row['refine_officialname_add4'],[])
    tmp_add5.append(row['CompanyID'])
    word_dict[row['refine_officialname_add4']] = tmp_add5
    for word in row['refine_officialname']:
        tmp = word_dict.get(word, [])
        tmp.append(row['CompanyID'])
        word_dict[word] = tmp

In [262]:
def predict(string):
    k = string.split()
    kk = [k[i:i+2] for i in range(len(k)-1)]
    kkk = [k[i:i+3] for i in range(len(k)-2)]
    kkkk = [k[i:i+4] for i in range(len(k)-3)]
    a = [' '.join(i) for i in kk]
    aa = [' '.join(i) for i in kkk]
    aaa = [' '.join(i) for i in kkkk]

    tmp1 = [word_dict.get(word.upper()) for word in k if word_dict.get(word.upper())]
    tmp2 = [word_dict.get(word.upper()) for word in a if word_dict.get(word.upper())]
    tmp3 = [word_dict.get(word.upper()) for word in aa if word_dict.get(word.upper())]
    tmp4 = [word_dict.get(word.upper()) for word in aaa if word_dict.get(word.upper())]

    tmp = sum(tmp1+tmp2+tmp3+tmp4,[])
    c_tmp = Counter(tmp)
    if len(c_tmp.most_common()) == 0:
        return []
    elif c_tmp.most_common()[0][1] == 1:
        return [c_tmp.most_common()[0][0]]
    else:
        return [item[0] for item in c_tmp.most_common()[:3] if item[1] !=1]

In [263]:
data['pred'] = data['refine_headline'].apply(predict)

[[1536]]
[[198, 222, 3430, 2408, 3371, 3133]]
[[1747, 789]]
[[3330], [1188, 2675]]
[[1844], [2171, 1679, 112, 1353, 1844, 765, 1205, 3564, 2961], [1844]]
[[3166], [3166]]
[[849, 849], [1995], [2331], [291], [291]]
[[1117]]
[[359], [359], [1314], [359]]
[[2012], [3211], [2708]]
[[739, 3721]]
[[3315], [1512]]
[[991], [1821, 991, 1619, 3169, 1246, 1267], [2192]]
[[3581], [3211]]
[[1614], [3466, 488, 1577, 1694, 3319, 1614, 2305, 2388, 1378], [3211], [1512]]
[[1059, 2376, 276, 1439, 441, 2494, 1383, 1719], [2540], [3461, 2943, 3673, 2807, 1583, 2540, 1414, 1886]]
[[2039, 2704], [1344, 1121, 2198, 2550, 3148, 1855]]
[[20], [2708]]
[[1066]]
[[1500], [1188, 2675]]
[[1049], [1669], [950, 93, 192, 283, 1355, 1354, 1037, 2823, 1047]]
[[279]]
[[620], [899], [1188, 2675]]
[[3165], [3165], [1314], [3165]]
[[1284]]
[]
[[2518], [1226, 16, 135, 136, 171, 1446, 651, 188, 1431]]
[[2284]]
[[254]]
[[54, 3356], [3329, 660]]
[[2179], [2078], [3445, 2854]]
[[2120]]
[[3337, 2471], [1704], [3415]]
[[3329, 660]

In [216]:
unique_company = company_map.CompanyID.values

## Jaccard similarity

In [217]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import jaccard_similarity_score

def Onehot(CompanyID, CompanyID_pred, unique):
    
    company_id = [i.split(",") for i in CompanyID]
    temp=[]; tmp=[];
    for i in company_id:
        k=[]
        for j in i:
            k.append(j.split(" "))
        temp.append(k)
    for i in temp:
        kk=[]
        for j in i:
            for l in j:
                kk.append(int(l))
        tmp.append(kk)
    pred = list(CompanyID_pred)

    mlb = MultiLabelBinarizer(classes=unique)
    onehot = mlb.fit_transform(tmp)
    onehot_pred = mlb.fit_transform(pred)

    return onehot, onehot_pred

onehot, pred = Onehot(data.CompanyID,data.pred,unique_company)
jaccard_similarity_score(onehot,pred)

0.6706166666666666

In [213]:
#[Headline]
# 0.3241428571428571   단어하나만 했을 때
# 0.33635952380952383  + 회사명에서 많이 나온 단어 빼고 나머지 단어 합친거 (hello word INC => hello, world, hello world)
# 0.3430761904761904   + 기사 본문이랑 제목에 많이 나오는 단어 빼고
# 0.5439595238095238   + most_common 3개 뽑고 1번 넘게 나온애들만 뽑음